### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
df_school_data = pd.read_csv(school_data_to_load)
df_student_data = pd.read_csv(student_data_to_load)

# Add student passing/fail to student data dataframe
student_passing_reading = []

for score in df_student_data['reading_score']:
    if score > 69:
        student_passing_reading.append(1)
    else:
        student_passing_reading.append(0)
        
df_student_data['student_passing_reading'] = student_passing_reading 

student_passing_math = []

for score in df_student_data['math_score']:
    if score > 69:
        student_passing_math.append(1)
    else:
        student_passing_math.append(0)
        
df_student_data['student_passing_math'] = student_passing_math 

student_overall_passing_nbr = df_student_data["student_passing_math"] + df_student_data["student_passing_reading"]
df_student_data["student_overall_passing"] = student_overall_passing_nbr
df_student_data['student_overall_passing'] = np.where(
   (df_student_data['student_overall_passing'] <=1), 0, 1)

df_school_data['budget_per_student'] = df_school_data["budget"] / df_school_data["size"]

school_spend_bins = [0, 584, 629, 644, 675]
school_spend_labels = ["< $584","$585-629","$630-644","$645-675"]

df_school_data["Spending Ranges (Per Student)"] = pd.cut(df_school_data["budget_per_student"], school_spend_bins, labels=school_spend_labels)

school_size_bins = [0, 999, 1999, 5000]
school_size_labels = ["Small < 1000","Medium (1000-1999)","Large (2000-5000)"]
df_school_data["School Size"] = pd.cut(df_school_data["size"], school_size_bins, labels=school_size_labels)

# Combine the data into a single dataset.  
df_school_data_complete = pd.merge(df_student_data, df_school_data, how="left", on=["school_name", "school_name"])


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#calculate variables
district_school_count = len(df_school_data_complete["school_name"].unique())
district_student_count = len(df_school_data_complete["Student ID"].unique())
district_total_budget = df_school_data["budget"].sum()
district_average_math_score = df_school_data_complete["math_score"].mean()
district_average_reading_score = df_school_data_complete["reading_score"].mean()

#get passing math score df
district_student_count_passing_math = df_school_data_complete["student_passing_math"].sum()
district_student_percent_passing_math = (district_student_count_passing_math / district_student_count) * 100

#get passing reading score df
district_student_count_passing_reading = df_school_data_complete["student_passing_reading"].sum()
district_student_percent_passing_reading = (district_student_count_passing_reading / district_student_count) * 100

#get overall passing both reading & math
district_overall_count = df_school_data_complete["student_overall_passing"].sum()
district_overall_percent = (district_overall_count/district_student_count) * 100

#Formatting
district_student_count = "{:,}".format(district_student_count)
district_total_budget = '${:,.2f}'.format(district_total_budget)
district_average_math_score = '{:,.2f}'.format(district_average_math_score)
district_average_reading_score = '{:,.2f}'.format(district_average_reading_score)
district_student_percent_passing_math = '{:,.2f}%'.format(district_student_percent_passing_math)
district_student_percent_passing_reading = '{:,.2f}%'.format(district_student_percent_passing_reading)
district_overall_percent = '{:,.2f}%'.format(district_overall_percent)

df_district_summary = pd.DataFrame({"Total Schools":[district_school_count],
                                 "Total Students":[district_student_count],
                                 "Total Budget":[district_total_budget],
                                 "Average Math Score":[district_average_math_score],
                                 "Average Reading Score":[district_average_reading_score],
                                 "% Passing Math":[district_student_percent_passing_math],
                                 "% Passing Reading":[district_student_percent_passing_reading],
                                 "% Overall Passing":[district_overall_percent]})

df_district_summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
df_grouped_school = df_school_data_complete.groupby(["school_name"])

school_type = df_grouped_school["type"].first()
school_student_count = df_grouped_school["Student ID"].count()
school_average_math_score = df_grouped_school["math_score"].mean()
school_average_reading_score = df_grouped_school["reading_score"].mean()
school_budget = df_grouped_school["budget"].sum()
school_percent_passing_math = df_grouped_school["student_passing_math"].sum()
school_percent_passing_math = (school_percent_passing_math/school_student_count)*100
school_percent_passing_reading = df_grouped_school["student_passing_reading"].sum()
school_percent_passing_reading = (school_percent_passing_reading/school_student_count)*100
school_overall_percent_passing = df_grouped_school["student_overall_passing"].sum()
school_overall_percent_passing = (school_overall_percent_passing/school_student_count)*100

school_budget = school_budget/school_student_count
school_per_student = school_budget/school_student_count
school_per_student_numeric = school_per_student

df_school_summary = pd.DataFrame({"School Type": school_type,
                                  "Total Students": school_student_count,
                                  "Total School Budget": school_budget,
                                  "Per Student Budget": school_per_student,
                                  "Average Math Score": school_average_math_score,
                                  "Average Reading Score": school_average_reading_score,
                                  "% Passing Math": school_percent_passing_math, 
                                  "% Passing Reading":school_percent_passing_reading,
                                  "% Overall Passing":school_overall_percent_passing})

format_mapping = {'Total School Budget': '${:,.2f}', 'Per Student Budget': '${:,.2f}','% Overall Passing': '{:.2f}%'}

for key, value in format_mapping.items():
    df_school_summary[key] = df_school_summary[key].apply(value.format)


df_school_summary


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.64%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.33%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.20%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.29%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.60%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.23%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.51%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.54%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [4]:
df_top_performers = df_school_summary
sort_by_top_performing = df_top_performers.sort_values(['% Overall Passing'],ascending=False)
sort_by_top_performing.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.33%
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.95%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.60%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.54%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [5]:
df_bottom_performers = df_school_summary
sort_by_bottom_performing = df_bottom_performers.sort_values(['% Overall Passing'])
sort_by_bottom_performing.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.99%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.20%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.51%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.53%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.54%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
df_scores_by_grade = df_school_data_complete

df_9_graders = df_scores_by_grade[(df_scores_by_grade["grade"] == "9th")]
df_9_grouped = df_9_graders.groupby(["school_name"])
avg_math_9th = df_9_grouped["math_score"].mean()

df_10_graders = df_scores_by_grade[(df_scores_by_grade["grade"] == "10th")]
df_10_grouped = df_10_graders.groupby(["school_name"])
avg_math_10th = df_10_grouped["math_score"].mean()

df_11_graders = df_scores_by_grade[(df_scores_by_grade["grade"] == "11th")]
df_11_grouped = df_11_graders.groupby(["school_name"])
avg_math_11th = df_11_grouped["math_score"].mean()

df_12_graders = df_scores_by_grade[(df_scores_by_grade["grade"] == "12th")]
df_12_grouped = df_12_graders.groupby(["school_name"])
avg_math_12th = df_12_grouped["math_score"].mean()

df_math_scores = pd.DataFrame({"9th": avg_math_9th,
                                  "10th": avg_math_10th,
                                  "11th": avg_math_11th,
                                  "12th": avg_math_12th})

df_math_scores

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
###################################
# USED SAME GROUP BY TABLES CREATED
# FOR MATH SCHOOL BY GRADE
###################################

avg_reading_9th = df_9_grouped["reading_score"].mean()
avg_reading_10th = df_10_grouped["reading_score"].mean()
avg_reading_11th = df_11_grouped["reading_score"].mean()
avg_reading_12th = df_12_grouped["reading_score"].mean()

df_reading_scores = pd.DataFrame({"9th": avg_reading_9th,
                                  "10th": avg_reading_10th,
                                  "11th": avg_reading_11th,
                                  "12th": avg_reading_12th})

df_reading_scores

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [8]:
df_grouped_per_student_spend = df_school_data_complete.groupby(["Spending Ranges (Per Student)"])

school_pss_student_count = df_grouped_per_student_spend["Student ID"].count()
school_pss_average_math_score = df_grouped_per_student_spend["math_score"].mean()
school_pss_average_reading_score = df_grouped_per_student_spend["reading_score"].mean()

school_pss_percent_passing_math = df_grouped_per_student_spend["student_passing_math"].sum()
school_pss_percent_passing_math = (school_pss_percent_passing_math/school_pss_student_count)*100
school_pss_percent_passing_reading = df_grouped_per_student_spend["student_passing_reading"].sum()
school_pss_percent_passing_reading = (school_pss_percent_passing_reading/school_pss_student_count)*100
school_pss_overall_percent_passing = df_grouped_per_student_spend["student_overall_passing"].sum()
school_pss_overall_percent_passing = (school_pss_overall_percent_passing/school_pss_student_count)*100


df_school_pss_summary = pd.DataFrame({"Average Math Score": school_pss_average_math_score,
                                  "Average Reading Score": school_pss_average_reading_score,
                                  "% Passing Math": school_pss_percent_passing_math, 
                                  "% Passing Reading":school_pss_percent_passing_reading,
                                  "% Overall Passing":school_pss_overall_percent_passing})

df_school_pss_summary

###########################################################################################
# Values do not match sample but according to analysis in excel the values shown below
# are correct
##########################################################################################
#           Average of math_score
#$585-629	79.98287335
#$630-644	77.82105635
#$645-675	77.04929749
#< $584	    83.36306533





,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
< $584,83.363065,83.964039,93.702889,96.686558,90.640704
$585-629,79.982873,82.312643,79.109851,88.513145,70.939239
$630-644,77.821056,81.301007,70.623565,82.600247,58.841194
$645-675,77.049297,81.005604,66.230813,81.109397,53.528791


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
df_grouped_school_size = df_school_data_complete.groupby(["School Size"])

school_size_student_count = df_grouped_school_size["Student ID"].count()
school_size_average_math_score = df_grouped_school_size["math_score"].mean()
school_size_average_reading_score = df_grouped_school_size["reading_score"].mean()

school_size_percent_passing_math = df_grouped_school_size["student_passing_math"].sum()
school_size_percent_passing_math = (school_size_percent_passing_math/school_size_student_count)*100
school_size_percent_passing_reading = df_grouped_school_size["student_passing_reading"].sum()
school_size_percent_passing_reading = (school_size_percent_passing_reading/school_size_student_count)*100
school_size_overall_percent_passing = df_grouped_school_size["student_overall_passing"].sum()
school_size_overall_percent_passing = (school_size_overall_percent_passing/school_size_student_count)*100


df_school_size_summary = pd.DataFrame({"Average Math Score": school_size_average_math_score,
                                  "Average Reading Score": school_size_average_reading_score,
                                  "% Passing Math": school_size_percent_passing_math, 
                                  "% Passing Reading":school_size_percent_passing_reading,
                                  "% Overall Passing":school_size_overall_percent_passing})

df_school_size_summary

###########################################################################################
# Values do not match sample but according to analysis in excel the values shown below
# are correct
##########################################################################################

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small < 1000,83.828654,83.974082,93.952484,96.040317,90.136789
Medium (1000-1999),83.372682,83.867989,93.616522,96.773058,90.624267
Large (2000-5000),77.477597,81.198674,68.652380,82.125158,56.574046


## Scores by School Type

* Perform the same operations as above, based on school type

In [10]:
df_grouped_school_type = df_school_data_complete.groupby(["type"])

school_type_student_count = df_grouped_school_type["Student ID"].count()
school_type_average_math_score = df_grouped_school_type["math_score"].mean()
school_type_average_reading_score = df_grouped_school_type["reading_score"].mean()

school_type_percent_passing_math = df_grouped_school_type["student_passing_math"].sum()
school_type_percent_passing_math = (school_type_percent_passing_math/school_type_student_count)*100
school_type_percent_passing_reading = df_grouped_school_type["student_passing_reading"].sum()
school_type_percent_passing_reading = (school_type_percent_passing_reading/school_type_student_count)*100
school_type_overall_percent_passing = df_grouped_school_type["student_overall_passing"].sum()
school_type_overall_percent_passing = (school_type_overall_percent_passing/school_type_student_count)*100


df_school_type_summary = pd.DataFrame({"Average Math Score": school_type_average_math_score,
                                  "Average Reading Score": school_type_average_reading_score,
                                  "% Passing Math": school_type_percent_passing_math, 
                                  "% Passing Reading":school_type_percent_passing_reading,
                                  "% Overall Passing":school_type_overall_percent_passing})

df_school_type_summary

###########################################################################################
# Values do not match sample but according to analysis in excel the values shown below
# are correct
##########################################################################################

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,90.560932
District,76.987026,80.962485,66.518387,80.905249,53.695878
